In [2]:
import os
import numpy as np
import pandas as pd

from astropy.modeling import blackbody as bb
from astropy.table import Table, Column, Row
from astropy import units as u

from lmfit import minimize, Parameters, fit_report

import pycoco as pcc
%matplotlib inline

# Extend LC

In [5]:
files = os.listdir(path + '/lc/')
files.remove('README')

for lc in files:
    print(lc)

    data = pd.read_csv(path + '/lc/'+lc, delim_whitespace=True, header=None, comment='#')
    data.columns = ['mjd', 'flux', 'flux_err', 'band']
    V = data.query('band == "BessellV"')

    if V.size > 0:
        V['flux'] /= 10
        V['flux_err'] *= np.sqrt(10)
        V['band'] = "swift_UVW1"

        data = pd.concat((data, V))
        data.to_csv(path + '/lc_extend/' + lc, header=False, sep=' ', index=False)


SN1996cb.dat


/Users/szymon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/szymon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/szymon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

SN2011dh.dat
SN2010as.dat
SN2012ap.dat
SN2007uy.dat
SN2013ge.dat
SN1993J.dat
SN2011fu.dat
SN2005bf.dat
iPTF13bvn.dat
SN1999dn.dat
SN2011ei.dat
SN2009iz.dat
SN2005hg.dat
SN2006aj.dat
SN2011bm.dat
SN1994I.dat
SN2009jf.dat
SN2008bo.dat
SN2007C.dat
SN2007Y.dat
SN2008ax.dat
SN2002ap.dat
SN2004aw.dat
SN2009er.dat
SN2003jd.dat
SN2007gr.dat
SN2008D.dat
SN1998bw.dat
SN2010al.dat
SN2009bb.dat


# Spectrum Extensions

In [6]:
params = Parameters()
params.add("T", value=10000, min=1000) 
params.add("flux_scale", value=1e23)
params.add("EBV", value=0.001, min=0.0) 


def bb_min_fun(params, filters, wavelength, verbose=False):
    T = params["T"]
    flux_scale = params["flux_scale"]
    EBV = params["EBV"]
    
    bb_flux = bb.blackbody_lambda(np.array(wavelength)*u.AA, temperature=T*u.Kelvin)

    bb_spec = pcc.classes.SpectrumClass()
    bb_spec.load_table(Table([wavelength, bb_flux], names=("wavelength", "flux")))

    bb_spec.flux = pcc.extinction.unred(bb_spec.wavelength, bb_spec.flux, EBV=EBV)

    bb_spec.data["flux"] = bb_spec.data["flux"] / flux_scale
    bb_spec.flux = bb_spec.flux / flux_scale

    bb_spec.get_specphot(filters, verbose=False)
    
    return bb_spec


def bb_min(params, specphot, filters, wavelength, verbose=False):
    bb_flux = bb_min_fun(params, filters, wavelength, verbose).specphot["flux"]

    residual = specphot["flux"] - bb_flux

    return residual

In [7]:
for snname in os.listdir(path + '/spec/'):
    if not os.path.exists(path + 'spec_extend/' + snname):
        os.mkdir(path + 'spec_extend/' + snname)
    
    sn = pcc.classes.SNClass(snname)

    sn.load_phot(path = os.path.join(pcc.defaults._default_data_dir_path, "lc/" + snname + ".dat"))
    sn.load_list(os.path.join(pcc.defaults._default_list_dir_path, snname + ".list"))
    sn.load_spec()
    sn.check_overlaps()

    for file_name, Spec in sn.spec.items():
        print(file_name)

        Spec.specphot = Table(names=("lambda_effective", "flux", "filter"), dtype=('f4', 'f4', 'S'))
        for filter_name in sn.phot.data_filters:
            flux = pcc.kcorr.calc_spectrum_filter_flux(filter_object=sn.phot.data_filters[filter_name], 
                                                       spectrum_object=Spec, verbose=False)

            Spec.specphot.add_row((sn.phot.data_filters[filter_name].lambda_effective, 
                                   flux, 
                                   filter_name))
            wav = sn.phot.data_filters[filter_name].wavelength
        
        out = minimize(bb_min, 
                       params, 
                       args=(Spec.specphot, sn.phot.data_filters, wav))
        best_bb = bb_min_fun(out.params, 
                             sn.phot.data_filters, 
                             Column(np.arange(2000, 11000)*u.Angstrom))
        
        w_red = np.where(best_bb.wavelength > np.nanmax(Spec.wavelength))
        
        idx = np.where(Spec.wavelength < np.min(Spec.wavelength) + 50)

        low = 0.2 * np.mean(Spec.flux)
        w_blue = np.arange(1500, np.min(Spec.wavelength))
        flux_blue = np.linspace(low, np.mean(Spec.flux[idx]), w_blue.size)
        
        new_flux = np.concatenate((flux_blue, Spec.flux, best_bb.flux[w_red]))
        new_wav = np.concatenate((w_blue, Spec.wavelength, best_bb.wavelength[w_red]))
        
        df = pd.DataFrame({'wav': new_wav, 'flux': new_flux})
        df.to_csv(path + 'spec_extend/' + snname + '/' + file_name, 
                  columns=['wav', 'flux'],
                  header=False, 
                  sep=' ', 
                  index=False)

2009bb_-4.85.txt
2009bb_-3.86.txt
2009bb_1.09.txt
2009bb_5.05.txt


/Users/szymon/anaconda/lib/python3.5/site-packages/scipy/integrate/quadrature.py:324: RuntimeWarning: divide by zero encountered in true_divide
  h0divh1 = h0 / h1
/Users/szymon/anaconda/lib/python3.5/site-packages/scipy/integrate/quadrature.py:326: RuntimeWarning: divide by zero encountered in true_divide
  y[slice1]*hsum*hsum/hprod +
/Users/szymon/anaconda/lib/python3.5/site-packages/scipy/integrate/quadrature.py:326: RuntimeWarning: invalid value encountered in multiply
  y[slice1]*hsum*hsum/hprod +
/Users/szymon/anaconda/lib/python3.5/site-packages/scipy/integrate/quadrature.py:326: RuntimeWarning: invalid value encountered in true_divide
  y[slice1]*hsum*hsum/hprod +
/Users/szymon/anaconda/lib/python3.5/site-packages/scipy/integrate/quadrature.py:327: RuntimeWarning: invalid value encountered in multiply
  y[slice2]*(2-h0divh1))
/Users/szymon/anaconda/lib/python3.5/site-packages/scipy/integrate/quadrature.py:326: RuntimeWarning: invalid value encountered in add
  y[slice1]*hsum*hs

2009bb_14.95.txt
2009bb_15.94.txt
2009bb_19.9.txt
2009bb_20.89.txt
2009bb_23.86.txt
2009bb_27.82.txt
2005hg_-12.54.txt
2005hg_-11.56.txt
2005hg_-9.6.txt
2005hg_-8.62.txt
2005hg_-7.64.txt
2005hg_-6.66.txt
2005hg_-5.68.txt
2005hg_-4.7.txt
2005hg_-3.72.txt
2005hg_-2.74.txt
2005hg_-1.76.txt
2005hg_-0.78.txt
2005hg_0.2.txt
2005hg_12.93.txt
2005hg_16.85.txt
2005hg_22.72.txt


/Users/szymon/anaconda/lib/python3.5/site-packages/pycocosn-0.9.19-py3.5.egg/pycoco/kcorr.py:247: RuntimeWarning: invalid value encountered in double_scalars
  return  integrated_flux/filter_area


ValueError: The input contains nan values